In [ ]:
!pip install rank_bm25
!pip install unidecode
!pip install --upgrade gensim
!pip install pandas
!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import string
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import gensim
from os import listdir
from os.path import isfile, join
from gensim.summarization.bm25 import BM25

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Abertura DataSets

In [4]:
mypath = './Datasets a serem usados/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [14]:
perguntas = []
for file in onlyfiles:
    df = pd.read_csv(mypath+file)
    perguntas.extend(df['perguntas'])

In [17]:
result = pd.DataFrame({'perguntas':perguntas})
print('Tamanho: {}'.format(result.shape[0]))

Tamanho: 13878


# Pre-processamento

In [18]:
perguntas = result['perguntas'].values
for i in range(perguntas.size):
    perguntas[i] = unidecode(perguntas[i]).lower()
result['perguntas'] = perguntas
result.head()

,perguntas
0,o covid-19 terminara em breve?
1,o covid-19 terminara?
2,quando covid-19 vai acabar?
3,quando o bloqueio covid-19 termina?
4,o covid-19 ira embora?


In [19]:
print(("Dados duplicados: %d")%(result.duplicated(['perguntas']).sum()))
result.drop_duplicates(['perguntas'],inplace=True)
result.reset_index(inplace= True,drop = True)
print("Shape dataset: {}".format(result.shape))

Dados duplicados: 1639
Shape dataset: (12239, 1)


# Salvando dataset

In [29]:
result.to_csv('./Datasets a serem usados/dataset_tratado.csv',index=False)

# Abrindo dataset

In [27]:
result = pd.read_csv('./Datasets a serem usados/dataset_tratado.csv')

perguntas = result['perguntas']

document = []
for pergunta,i in zip(perguntas,range(perguntas.size)):
    try:
        tokens = nltk.word_tokenize(pergunta,language = 'portuguese')
        tokens = [w for w in tokens if not w in string.punctuation]
        document.append(tokens)
    except:
        print(pergunta, i)
document = np.array(document,dtype=object)

# Salvando amostra

In [22]:
#sample = np.random.choice(document,size=500,replace=False)

In [23]:
#with open('./Amostra Analise Dataset/amostra.json','w') as outfile:
#    json.dump(sample.tolist(),outfile)

# Abrindo amostra

In [49]:
with open('./Amostra Analise Dataset/amostra.json','r') as f:
    sample = json.load(f)

# Utilizando BM25

In [50]:
bm25 = BM25(document)

In [51]:
posicao =  10
try:
    query = sample[posicao]
    if("?" not in query):
        print('Query:'," ".join(query) + '?','\n')
    else:
        print(" ".join(query),'\n')
    score = bm25.get_scores(query)
    top_n = np.argsort(score)[::-1][2:5] # Apresentar as 3 melhores assimilações
    for i in top_n:
        print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25
except:
    print("Posicao invalida")

Query: casos de coronavirus na fl? 

Pergunta: casos confirmados de coronavirus em fl | Score BM25: 11.23
Pergunta: quantos casos de coronavirus em orlando fl | Score BM25: 10.52
Pergunta: quantos casos de coronavirus em napoles fl | Score BM25: 10.52


# Merge resultados avaliados

In [4]:
avaliador = [[100, 0, 0], [100, 100, 100], [100, 100, 100], [0, 0, 0], [100, 100, 100], [100, 0, 0], [100, 80, 0], [0, 0, 0], [50, 0, 100], [100, 0, 0], [100, 0, 0], [20, 0, 0], [100, 100, 100], [100, 100, 20], [50, 50, 0], [100, 0, 100], [80, 80, 0], [80, 100, 100], [0, 100, 100], [100, 40, 40], [100, 100, 100], [100, 0, 100], [100, 100, 100], [0, 0, 80], [100, 100, 100], [100, 100, 100], [100, 30, 30], [0, 0, 100], [100, 0, 50], [100, 100, 100], [100, 100, 100], [100, 0, 100], [0, 0, 0], [0, 0, 0],[0, 0, 0], [100, 0, 0], [0, 0, 70], [100, 100, 0], [100, 100, 100], [100, 100, 0], [80, 70, 70], [70, 70, 0], [0, 0, 70], [80, 70, 80], [0, 50, 80], [0, 0, 70], [60, 30, 40], [0, 80, 100], [100, 20, 0], [0, 0, 0], [100, 100, 100], [100, 100, 100], [0, 50, 100], [100, 80, 80], [0, 0, 0], [0, 0, 30], [40, 40, 75], [90, 0, 0], [100, 100, 100], [100, 100, 100], [100, 100, 50], [40, 100, 90], [0, 80, 40], [100, 100, 0], [80, 0, 40], [10, 0, 0], [0, 70, 0], [0, 100, 0], [100, 100, 0], [100, 100, 100], [0, 0, 40], [40, 40, 0], [0, 0, 0], [0, 0, 0], [100, 100, 100], [60, 60, 60], [0, 0, 0], [0, 40, 80], [0, 0, 0], [100, 100, 90], [80, 90, 0], [0, 0, 0], [80, 80, 0], [30, 50, 0], [90, 80, 90], [0, 0, 0], [0, 0, 0], [80, 80, 90], [0, 0, 100], [0, 0, 0], [100, 0, 0], [90, 90, 0],[0, 0, 0], [100, 100, 100], [30, 0, 30], [80, 100, 0], [0, 40, 0], [40, 40, 40], [100, 100, 100], [100, 30, 100],[0, 100, 80], [0, 0, 0], [70, 0, 70], [80, 80, 80], [70, 80, 40], [80, 0, 0], [0, 75, 0], [70, 80, 10], [70, 70, 70], [40, 40, 0], [100, 100, 100], [100, 0, 0], [0, 80, 0], [0, 0, 70], [100, 80, 0], [75, 0, 90], [80, 80, 70], [0, 0, 0], [0, 0, 0], [0, 80, 80], [80, 80, 80], [90, 75, 75], [80, 75, 80], [80, 80, 80], [100, 100, 60], [100, 45, 0], [75, 80, 80], [80, 90, 80], [90, 0, 40], [0, 0, 0], [90, 80, 90], [0, 0, 0], [45, 45, 0], [80, 80, 0], [0, 0, 0], [0, 80, 80], [90, 60, 30], [75, 75, 75], [40, 0, 0], [0, 0, 0], [90, 80, 100], [0, 0, 0], [100, 100, 100], [90, 90, 90], [65, 0, 0], [0, 90, 0], [0, 0, 0], [90, 80, 90], [90, 0, 0], [90, 90, 90], [80, 90, 90],[80,10,80],[100, 0, 100], [100, 0, 100], [90, 80, 100], [0, 80, 0], [0, 40, 0], [100, 100, 100], [0, 100, 0], [0, 0, 0], [0, 90, 30], [30, 30, 0], [0, 0, 70], [0, 0, 75], [0, 0, 90], [30, 0, 0], [0, 0, 0], [0, 0, 0], [90, 0, 0], [60, 90, 90], [90, 90, 15], [0, 45, 80], [0, 15, 0], [0, 0, 0], [90, 90, 80], [0, 0, 80], [0, 0, 0], [0, 0, 0], [100, 45, 45], [45, 45, 35], [90, 15, 90], [0, 90, 100], [90, 15, 0], [25, 25, 25], [80, 0, 0], [100, 25, 100], [80, 80, 80], [100, 100, 100], [100, 100, 100], [45, 90, 703], [65, 65, 65], [0, 0, 0], [0, 0, 80], [90, 0, 45], [0, 0, 0], [90, 90, 90], [80, 0, 0], [0, 75, 0], [45, 0, 0], [80, 0, 0], [80, 0, 75], [90, 90, 90], [90, 90, 0], [80, 0, 75], [75, 75, 80], [0, 75, 80], [0, 15, 0], [80, 55, 0], [75, 0, 0], [65, 80, 80], [75, 65, 75], [45, 75, 0], [90, 75, 0], [100, 90, 80], [100, 100, 100], [0, 100, 0], [100, 100, 100], [0, 0, 0], [90, 90, 80], [0, 65, 0], [80, 80, 80], [80, 80, 0], [0, 0, 0], [65, 65, 75], [0, 90, 0], [100, 90, 80], [100, 80, 90], [0, 65, 85], [0, 0, 0], [65, 75, 75], [90, 90, 75], [100, 100, 100], [100, 75, 100], [45, 35, 15], [75, 75, 75], [45, 65, 80], [90, 0, 0], [80, 80, 80], [60, 80, 0], [0, 35, 45], [80, 80, 80], [0, 0, 0], [90, 35, 80], [0, 80, 80], [0, 80, 80], [0, 0, 0], [35, 35, 90], [90, 0, 90], [65, 65, 65], [0, 0, 0], [35, 80, 80], [90, 90, 75]]

In [5]:
result = pd.read_csv('./Datasets a serem usados/dataset_tratado.csv')

perguntas = result['perguntas']

document = []
for pergunta,i in zip(perguntas,range(perguntas.size)):
    try:
        tokens = nltk.word_tokenize(pergunta,language = 'portuguese')
        tokens = [w for w in tokens if not w in string.punctuation]
        document.append(tokens)
    except:
        print(pergunta, i)
document = np.array(document,dtype=object)

with open('./Amostra Analise Dataset/amostra.json','r') as f:
    sample = json.load(f)

bm25 = BM25(document)

aux = []
k = 0
while(k < 252):

    posicao =  k
    try:
        query = sample[posicao]
        score = bm25.get_scores(query)
        top_n = np.argsort(score)[::-1][2:5] # Apresentar as 3 melhores assimilações
        
        aux.append([' '.join(query),perguntas[top_n[0]],avaliador[posicao][0],perguntas[top_n[1]],avaliador[posicao][1],perguntas[top_n[2]],avaliador[posicao][2]])
        k += 1
    except Exception as x:
        print(str(x))
        print("Erro")
        break

In [6]:
pd.DataFrame(aux,columns=['query','per_um','ava_um','per_dois','ava_dois','per_tres','ava_tres']).to_csv('./gabarito_parcial.csv',index=False)

In [14]:
a_2 = pd.DataFrame(aux,columns=['query','per_um','ava_um','per_dois','ava_dois','per_tres','ava_tres'])

In [10]:
query = sample[252]
score = bm25.get_scores(query)
top_n = np.argsort(score)[::-1][2:5]
for i in top_n:
    print(perguntas[i])

melhores maneiras de manter seu sistema imunologico saudavel
maneiras de manter o sistema imunologico saudavel
maneiras de manter um sistema imunologico saudavel


In [11]:
aaaa= [["maneiras de manter seu sistema imunologico saudavel", "melhores maneiras de manter seu sistema imunologico saudavel", 90, "maneiras de manter o sistema imunologico saudavel", 100, "maneiras de manter um sistema imunologico saudavel", 100], ["covid-19 e covid-19 sao iguais", "como covid-19 e sars cov sao semelhantes?", 80, "quais sao as diferencas entre influenza e covid-19?", 40, "os testes covid-19 sao precisos?", 0], ["o que fortalece seu sistema imunologico", "que vitamina fortalece seu sistema imunologico", 35, "que comida fortalece o seu sistema imunologico", 35, "que comida fortalece o sistema imunologico", 35], ["todos os empregos voltarao rapidamente quando a pandemia covid-19 acabar", "o que precisa mudar quando a pandemia covid-19 acabar?", 0, "quando covid-19 acabar?", 45, "que setores criariam empregos apos a pandemia covid-19?", 50], ["e o virus corona na nigeria", "http // e o virus corona na nigeria", 90, "noticias do virus corona na nigeria", 90, "atualizacao do virus corona na nigeria", 90], ["como os sintomas do coronavirus comecam", "como os sintomas do coronavirus", 70, "quais os sintomas do coronavirus?", 65, "quais os sintomas do covid-19?", 65], ["posso continuar minhas atividades ao ar livre por exemplo corro na praia ou no parque posso continuar neste momento", "coronavirus posso ir ao parque", 80, "coronavirus, posso ir ao parque reino unido?", 75, "e arriscado continuar trabalhando?", 0], ["os sintomas de covid-19 aparecem rapidamente", "os sintomas de covid-19 aparecem rapido?", 100, "quais sintomas de covid-19 aparecem primeiro?", 50, "quanto tempo apos a exposicao os sintomas do coronavirus aparecem", 50], ["o que significa se voce tem um sistema imunologico fraco", "o que significa quando voce tem um sistema imunologico fraco", 100, "o que significa se voce tem um sistema imunologico ruim", 100, "o que significa se voce tem um sistema imunologico baixo", 100], ["havera medida de restricao ou bloqueio a pessoascom sintomas vindas de outras cidades", "existe alguma medida de monitoramento de temperatura de pessoas vindas da china ou de outro pais com registro do covid-19?", 70, "havera um covid-19 20 ou superior?", 0, "adicionar pimenta a sopa ou outras refeicoes pode ajudar a prevenir ou curar o covid-19?", 0], ["ultimas noticias sobre vacina contra coronavirus na india", "ultimas noticias sobre coronavirus na india", 100, "ultimas noticias sobre virus corona na india", 100, "ultimas noticias na india sobre virus corona", 100], ["os pacientes com covid-19 devem tomar advil", "os pacientes com covid-19 devem ser intubados?", 0, "os pacientes com covid-19 devem ser ventilados?", 0, "os pacientes com covid-19 devem evitar o ibuprofeno?", 30], ["como voce pode manter seu sistema imunologico saudavel", "como voce pode manter um sistema imunologico saudavel", 100, "como manter seu sistema imunologico saudavel", 100, "o que voce pode fazer para manter seu sistema imunologico saudavel", 100], ["como seu corpo luta contra o virus da gripe", "como seu corpo luta contra virus", 60, "como seu corpo luta contra o virus", 60, "como seu corpo luta contra os virus", 60], ["coronavirus originado", "coronavirus originado de onde", 100, "coronavirus originado dos eua", 0, "coronavirus originado de nos", 0], ["coronavirus confirmado nas filipinas", "coronavirus nas filipinas", 70, "coronavirus nas filipinas 2020", 70, "coronavirus nas noticias filipinas", 80], ["como covid-19 comeca", "qual covid-19 comeca?", 70, "onde covid-19 comeca?", 30, "como o coronavirus comeca", 100], ["coronavirus na coreia ultimas noticias", "virus corona na coreia ultimas noticias", 100, "ultimas noticias corona virus coreia", 100, "corona virus ultimas noticias coreia", 100], ["laboratorio de origem de coronavirus", "e coronavirus de um laboratorio", 80, "covid-19 veio de um laboratorio?", 80, "mercado de origem de coronavirus", 0], ["o virus corona e um virus rna ou dna", "o virus corona e um virus rna ou dna", 100, "e o virus corona rna ou virus dna", 100, "e o virus corona rna ou dna", 100], ["coronavirus hku1 pcr", "coronavirus hku1 tailandia", 0, "coronavirus hku1 deutsch", 0, "coronavirus hku1 2020", 70], ["novo caso de coronavirus nos eua", "qualquer caso de coronavirus nos eua", 70, "primeiro caso de coronavirus nos eua", 45, "coronavirus nos eua primeiro caso", 45], ["quanto tempo dura o coronavirus em um cdc de superficie", "quanto tempo dura o coronavirus cdc", 0, "quanto tempo o coronavirus dura em uma superficie", 80, "quanto tempo dura o coronavirus em um objeto", 80], ["coronavirus iniciado por", "coronavirus iniciado por morcegos", 70, "virus corona iniciado por", 80, "qual e o coronavirus iniciado por", 80], ["como impulsionar o sistema imunologico apos antibioticos", "melhor maneira de impulsionar o sistema imunologico apos antibioticos", 100, "como reconstruir o sistema imunologico apos antibioticos", 10, "como aumentar o sistema imunologico apos antibioticos", 100], ["quais devem ser os cuidados com os bebes", "os pacientes com covid-19 devem ser intubados?", 0, "os pacientes com covid-19 devem ser ventilados?", 0, "quais cuidados se deve ter com os gatos?", 0], ["atualizacao de coronavirus para os eua", "atualizacao de coronavirus nos eua", 100, "atualizacao de casos de coronavirus eua", 100, "coronavirus se espalhou para os eua", 45], ["coronavirus em nyc chinatown", "coronavirus nyc chinatown confirmado", 80, "coronavirus em nyc", 90, "coronavirus confirmado em nyc", 80], ["como construir o sistema imunologico durante a quimio", "como fortalecer meu sistema imunologico durante a quimio", 100, "como construir seu sistema imunologico apos a quimio", 100, "mantendo o sistema imunologico forte durante a quimio", 100], ["coronavirus no quirguistao ultimas noticias", "coronavirus no quirguistao bishkek", 80, "coronavirus ultimas noticias no paquistao", 0, "ultimas noticias sobre coronavirus no canada", 0], ["como construir o sistema imunologico quando doente", "como aumentar o sistema imunologico quando doente", 100, "como melhorar o sistema imunologico quando doente", 100, "como impulsionar o sistema imunologico quando doente", 100], ["casos de coronavirus na africa", "casos de coronavirus na africa do sul", 10, "quantos casos de coronavirus na africa do sul", 70, "casos de virus corona na africa do sul", 90], ["quantos casos de coronavirus em augusta ga", "casos de coronavirus em ga", 70, "coronavirus em ga", 90, "quantas mortes por coronavirus em ga", 80], ["em que o virus corona animal encontrado", "e o virus corona encontrado em deli", 0, "e o virus corona encontrado em frango", 0, "e o virus corona encontrado em galinhas em bangalore", 0], ["como o coronavirus se compara a gripe", "como o coronavirus se compara a praga", 10, "como o coronavirus se compara", 50, "como o covid-19 se compara a outros virus?", 70], ["como voce deixa seu sistema imunologico mais forte", "como voce pode tornar seu sistema imunologico mais forte", 100, "como seu sistema imunologico fica mais forte", 100, "como tornar seu sistema imunologico mais forte", 100], ["como ter certeza de que seu sistema imunologico e forte", "como ter um sistema imunologico forte e saudavel", 60, "como posso ter certeza de que meu filho continua recebendo sua educacao?", 0, "como dizer que seu sistema imunologico e forte", 90], ["quanto tempo leva para o virus corona mostrar os sintomas", "quanto tempo leva para o virus corona mostrar os sintomas", 100, "quanto tempo o coronavirus leva para mostrar os sintomas", 100, "quanto tempo leva o coronavirus para mostrar os sintomas", 100], ["quais medidas estao sendo tomadas para proteger os eua suprimento de sangue do virus que causa covid-19", "quais sao as medidas de seguranca tomadas globalmente para prevenir covid-19?", 0, "que medidas os pais devem tomar para proteger os filhos?", 0, "por quais razoes os funcionarios do hospital estao sendo expostos ao covid-19?", 0], ["mapa de infeccao do virus corona", "mapa do virus corona de casos americanos", 40, "mapa de infeccao covid-19 eua", 40, "dados gis do mapa de infeccao por coronavirus", 80], ["que medidas devem ser tomadas para garantir que agua potavel seja usada na producao de alimentos durante esta pandemia", "como as instalacoes de alimentos devem ser limpas / desinfetadas durante esta pandemia?", 50, "que medidas adicionais devem ser tomadas nos locais de trabalho e para empregos de alto risco?", 10, "que medidas adicionais devem ser tomadas nos locais de trabalho e para empregos de medio risco?", 10], ["quais sao os sintomas mais comuns de covid-19", "quais sao alguns equivocos comuns sobre covid-19?", 0, "quais sao os sintomas?", 70, "quais sao os sintomas da covid-19", 70], ["tratamento com coronavirus hku1", "tratamento com medicamento para coronavirus", 50, "coronavirus hku1 rna", 0, "coronavirus hku1 2020", 0], ["vacina contra coronavirus romenia", "vacina contra coronavirus", 60, "e coronavirus na romenia", 80, "vacina contra cinomose canina contra sarampo", 0], ["coronavirus na florida ultimas noticias", "coronavirus na malasia ultimas noticias", 0, "coronavirus na alemanha ultimas noticias", 0, "coronavirus na espanha ultimas noticias", 0], ["o que e bom para um sistema imunologico saudavel", "o que e um bom suplemento para o sistema imunologico", 60, "o que levar para um sistema imunologico saudavel", 80, "o que comer para um sistema imunologico saudavel", 80], ["coronavirus posso andar la fora", "coronavirus posso correr la fora", 100, "coronavirus voce pode ir la fora", 80, "o coronavirus esta no ar la fora", 80], ["o lisol diz que mata o coronavirus", "o que e coronavirus humano em lisol", 70, "o que a fe islamica diz sobre covid-19?", 0, "e coronavirus em lisol", 80], ["e verdade que 5g espalha o covid-19", "e verdade que aqueles que receberam vacinacao bcg teriam imunidade contra covid-19?", 0, "por que tantas pessoas acreditam que ha uma conexao entre a tecnologia covid-19 e 5g?", 100, "e verdade que os tipos de sangue o + sao imunes ao covid-19?", 0], ["o que posso fazer se meu filho tiver dificuldades para se ajustar a novas rotinas e seguir as recomendacoes", "posso fazer meu proprio desinfetante para as maos?", 0, "o que posso fazer se meu sistema imunologico estiver fraco", 0, "posso tocar e segurar meu bebe recem-nascido se eu tiver covid-19?", 0], ["o covid-19 ira embora", "o covid-19 ira embora no verao?", 60, "o covid-19 ira embora na primavera?", 60, "o covid-19 algum dia ira embora?", 100], ["covid-19 e curavel", "coronavirus e curavel", 100, "e coronavirus curavel em humanos", 100, "coronavirus e curavel ou nao", 100], ["vacina contra coronavirus para caes", "vacina contra coronavirus em caes", 100, "vacina contra cinomose canina em caes", 0, "vacina de coronavirus para caes", 100], ["casos confirmados de coronavirus em arkansas", "onde estao os casos de coronavirus em arkansas", 90, "e coronavirus em arkansas", 80, "casos confirmados de coronavirus em nos", 0], ["como manter seu sistema imunologico forte e saudavel", "como manter o sistema imunologico forte e saudavel", 100, "como manter seu sistema imunologico saudavel", 80, "como manter seu sistema imunologico forte", 80], ["quais alimentos aumentam meu sistema imunologico", "quais alimentos aumentam o sistema imunologico", 100, "quais alimentos aumentam naturalmente o seu sistema imunologico", 100, "quais alimentos e bebidas aumentam seu sistema imunologico", 100], ["virus corona da india confirmado", "virus corona da india mumbai", 0, "bangalore virus corona da india", 0, "virus corona da india afetado", 70], ["a oms recomenda o uso de mascaras com valvulas expiratorias para prevenir a transmissao de covid-19", "a oms recomenda o uso de dexametasona para pacientes com covid-19?", 0, "a oms recomenda o uso de mascaras de rotina para pessoas saudaveis   durante o surto de ncov de 2019?", 0, "o que a oms recomenda para usuarios de tabaco?", 0], ["novo mapa de infeccao por coronavirus", "novo tratamento para infeccao por coronavirus", 0, "dados gis do mapa de infeccao por coronavirus", 80, "mapa do pais com infeccao por coronavirus", 80], ["existe alguma orientacao especifica para gestantes que comecem a apresentar sintomas de covid-19", "existe alguma cura para covid-19?", 0, "existe uma mascara especifica que devo usar para prevenir covid-19?", 0, "existe alguma injecao de desinfetante disponivel para covid-19?", 0], ["quantos casos de coronavirus em west hollywood", "quantos casos de coronavirus em de", 0, "quantos casos de coronavirus em taiwan", 0, "quantos casos de coronavirus em ny", 0], ["que alternativas eu tenho se nao houver desinfetantes para as maos disponiveis", "se eu nao conseguir encontrar desinfetante para as maos na loja, posso fazer o meu?", 100, "eu tenho permissao para sair?", 0, "devo fazer meu proprio desinfetante para as maos se nao conseguir encontra-lo nas lojas?", 100], ["por quanto tempo voce vomita com norovirus", "por quanto tempo voce pode ser contagioso com norovirus", 0, "quanto tempo voce permanece contagioso com norovirus", 0, "quanto tempo voce fica contagioso com norovirus", 0], ["o coronavirus afetara a industria da construcao", "o coronavirus esta afetando a construcao", 100, "o coronavirus afetara a internet", 0, "o coronavirus afetara a voz", 0], ["covid-19 esta minando nossa liberdade", "como o covid-19 afetara nossa geracao?", 20, "o covid-19 mudara a nossa politica?", 0, "coronavirus e nossa patente", 10], ["o coronavirus esta espalhado pelo ar", "o coronavirus esta espalhado no ar", 80, "coronavirus espalhado pelo dinheiro", 0, "coronavirus espalhado pelo correio", 0], ["paquistao coronavirus 4 casos", "casos de coronavirus no paquistao", 80, "casos de coronavirus noticias no paquistao", 80, "casos confirmados de coronavirus no paquistao", 80], ["quantos casos confirmados de coronavirus em nos", "casos confirmados de coronavirus em nos", 100, "quantos casos confirmados de coronavirus em idaho", 0, "quantos casos confirmados de coronavirus em maine", 0], ["por que os negros americanos estao morrendo em maior numero desde covid-19", "por que os trabalhadores medicos estao adoecendo ou morrendo de covid-19 se eles estao usando equipamentos de protecao?", 0, "por que os americanos sao a nacao mais afetada do mundo por covid-19?", 50, "por que os politicos mundiais estao subestimando covid-19?", 0], ["coreia do sul covid localizacao de 19 casos", "covid 19 coreia do sul", 70, "covid 19 casos da coreia do sul", 80, "covid 19 coreia do sul cingapura", 0], ["corona virus quanti casi na germania", "coronavirus quanti casi na italia", 0, "quanti casi no virus sicilia di corona", 0, "em piemonte quanti casi di corona virus ci sono", 0], ["voce esta entediado de ficar em casa devido a covid-19", "por quanto tempo o publico americano esta disposto a ficar em casa devido a covid-19?", 70, "a china ainda esta bloqueada devido a covid-19?", 0, "devemos ficar em casa para nos proteger?", 40], ["posso passar alcool gel na mao de recem-nascidos e criancas", "diante da falta de alcool gel podemos colocar gel de cabelo no alcool 70 liquido como alternativa?", 0, "o alcool 92% e eficiente? e o alcool de cereais 93%, eles tem a mesma funcao do alcool gel 70%?", 0, "o alcool em gel substitui a lavagem com agua e sabao?", 50], ["status do coronavirus no ira", "ultimas noticias do virus corona no ira", 80, "casos de coronavirus no ira", 80, "status do coronavirus na india", 0], ["voce pode ter dois virus ao mesmo tempo", "quanto tempo voce pode ter norovirus", 0, "quanto tempo voce pode ter o norovirus", 0, "por quanto tempo voce pode ter norovirus por", 0], ["noticias falsas sobre coronavirus india", "noticias sobre coronavirus na india", 80, "ultimas noticias sobre coronavirus na india", 80, "noticias sobre coronavirus india em marati", 40], ["e se meu filho precisar ir ao hospital", "coronavirus e seguro ir ao parque", 0, "coronavirus e seguro ir ao supermercado", 0, "se nao ha cura, por que ir ao hospital a menos que tenha problemas respiratorios?", 0], ["foi o homem covid-19 feito", "covid-19 e um virus feito pelo homem?", 100, "coronavirus e feito pelo homem", 100, "coronavirus feito pelo homem hiv", 70], ["mapa-mundi do contador em tempo real da pandemia de coronavirus", "pandemia de coronavirus em tempo real", 80, "contador de pandemia de coronavirus", 90, "contagem de mapas mundiais em tempo real de pandemia de coronavirus", 100], ["quantos casos de coronavirus em israel", "casos de coronavirus em israel hoje", 100, "e coronavirus em israel", 100, "vacina de coronavirus israel", 0], ["vacina para o coronavirus", "existe vacina para covid-19?", 100, "existe uma vacina para o coronavirus?", 100, "vacina de coronavirus para caes", 50], ["como manter o sistema imunologico forte durante a gravidez", "como manter o sistema imunologico ativo durante a gravidez", 100, "como manter seu sistema imunologico saudavel durante a gravidez", 100, "como manter o sistema imunologico forte durante a quimioterapia", 0], ["coronavirus em condados de michigan", "coronavirus em quarentena de michigan", 30, "coronavirus em casos de michigan", 70, "casos de coronavirus em michigan", 50], ["o coronavirus pode causar nauseas", "pode o coronavirus causar tontura", 90, "pode o coronavirus causar vomito", 30, "o coronavirus pode causar nausea", 100], ["o virus corona se espalha pelo frango", "o coronavirus se espalha pelo frango", 100, "o virus corona se espalha pelo ovo", 50, "o coronavirus pode se espalhar pelo frango", 100], ["onde posso fazer os testes rapidos igm/igg", "o que sao testes rapidos (igm/igg)?", 30, "e os testes rapidos?", 50, "o que sao os testes rapidos?", 20], ["por que as pessoas estao mais preocupadas em voltar ao trabalho do que em ficar em casa ate que o virus covid-19 melhore", "os trabalhadores da alimentacao que estao doentes devem ficar em casa?", 0, "o que significa os governadores ficar em casa?", 0, "quanto tempo as pessoas com covid-19 devem ficar em casa e isoladas?", 0], ["quantos casos do coronavirus existem nos estados unidos", "quantos casos do coronavirus nos estados unidos", 100, "quantos casos de coronavirus nos estados unidos", 100, "quantos casos do coronavirus foram encontrados nos estados unidos", 100], ["se o resultado do teste rapido igm/igg for positivo isso indica que tenho covid-19", "que teste devo fazer para ver se tenho covid-19?", 0, "quem fez teste rapido em laboratorios nao corre um alto risco de ter recebido um resultado falso positivo ou negativo? os testes rapidos (igm/igg) sao confiaveis?", 35, "o que aconteceria se o teste de trunfo fosse positivo para covid-19?", 0], ["coronavirus e chicago", "coronavirus china chicago", 0, "coronavirus novo chicago", 70, "coronavirus chicago univision", 35], ["qual virus corona animal espalhou", "virus corona espalhado por qual animal", 0, "virus corona e transmitido de qual animal", 0, "qual animal e a causa do virus corona", 35], ["quanto tempo leva para as pessoas infectadas apresentarem os sintomas", "por quanto tempo as pessoas permanecem infectadas com covid-19?", 0, "virus corona quanto tempo leva para mostrar os sintomas", 100, "quanto tempo leva o coronavirus para mostrar os sintomas", 100], ["como lidar com um sistema imunologico fraco", "como se livrar de um resfriado com um sistema imunologico fraco", 10, "como curar um sistema imunologico fraco", 90, "como testar um sistema imunologico fraco", 0], ["covid 19 malaysia caso 26", "covid 19 malaysia uda", 25, "covid 19 malaysia berita", 25, "covid 19 malaysia moh", 25], ["o coronavirus teve origem no laboratorio do governo chines", "o coronavirus se originou no laboratorio chines", 100, "laboratorio de origem de coronavirus", 80, "coronavirus no mercado chines", 0], ["atualizacoes de coronavirus nos", "atualizacoes de coronavirus nj", 0, "atualizacoes de coronavirus na california", 0, "atualizacoes do coronavirus nyc", 0], ["prevencao do coronavirus alemanha", "prevencao do coronavirus malasia", 0, "atualizacao do coronavirus na alemanha", 35, "prevencao do virus corona na india", 0], ["e coronavirus em perth", "e coronavirus em maryland", 0, "e coronavirus em vancouver", 0, "e coronavirus em michigan", 0], ["o que levar para um sistema imunologico saudavel", "o que levar para o sistema imunologico baixo", 0, "o que levar para o sistema imunologico fraco", 0, "o que levar para aumentar o sistema imunologico", 90], ["como o virus corona afeta o ser humano", "como o coronavirus afeta o corpo humano", 100, "como o virus corona afeta o corpo", 100, "como o virus corona afeta os humanos", 100], ["as vacinas covid-19 serao eficazes uma vez que uma pessoa recuperada pode contrair o virus novamente", "pode uma pessoa jovem e saudavel morrer de uma infeccao covid-19?", 0, "voce pode pegar um covid-19 mais de uma vez?", 60, "o coronavirus pode matar uma pessoa saudavel", 0], ["onde esta o coronavirus agora mapa", "onde esta o coronavirus agora nos eua", 35, "onde esta o coronavirus nos eua agora", 35, "onde esta o mapa do coronavirus", 90], ["mortes por coronavirus no reino unido ontem", "mortes por coronavirus no reino unido por idade", 65, "mortes por coronavirus no reino unido por dia", 70, "mortes por coronavirus no reino unido por regiao", 70], ["o que e coronavirus humano em lisol", "o que e coronavirus humano", 0, "o lisol diz que mata o coronavirus", 10, "e coronavirus na lata de lisol", 80]]

In [15]:
a_1 = pd.DataFrame(aaaa,columns=['query','per_um','ava_um','per_dois','ava_dois','per_tres','ava_tres'])

In [19]:
df = pd.concat([a_2,a_1])

In [21]:
df.reset_index(drop=True,inplace=True)

In [24]:
df.to_csv('./gabarito_parcial.csv',index=False)

# Merge resultados avaliados etapa 2

In [6]:
df = pd.read_csv('gabarito_parcial.csv')
df

,query,per_um,ava_um,per_dois,ava_dois,per_tres,ava_tres
0,o covid-19 acabara com o mundo,"assumindo que covid-19 nao acabou, isso acabar...",100,o covid-19 algum dia acabara?,0,que licoes o mundo aprendera com covid-19?,0
1,o que e covid 19 malaysia,covid 19 malaysia puchong,100,covid 19 malaysia berita,100,covid 19 malaysia moh,100
2,onde esta o coronavirus reino unido,onde esta o coronavirus agora no reino unido,100,"coronavirus no reino unido, onde",100,casos de coronavirus no reino unido onde,100
3,como as familias podem se preparar para o bloq...,como se preparar para o coronavirus nos eua,0,o que os idosos podem fazer para se preparar p...,0,como a oms esta ajudando os paises a se prepar...,0
4,quantos casos de coronavirus na fl,quantos casos de coronavirus em orlando fl,100,quantos casos de coronavirus em jacksonville fl,100,quantos casos de coronavirus em napoles fl,100
...,...,...,...,...,...,...,...
351,como o virus corona afeta o ser humano,como o coronavirus afeta o corpo humano,100,como o virus corona afeta o corpo,100,como o virus corona afeta os humanos,100
352,as vacinas covid-19 serao eficazes uma vez que...,pode uma pessoa jovem e saudavel morrer de uma...,0,voce pode pegar um covid-19 mais de uma vez?,60,o coronavirus pode matar uma pessoa saudavel,0
353,onde esta o coronavirus agora mapa,onde esta o coronavirus agora nos eua,35,onde esta o coronavirus nos eua agora,35,onde esta o mapa do coronavirus,90
354,mortes por coronavirus no reino unido ontem,mortes por coronavirus no reino unido por idade,65,mortes por coronavirus no reino unido por dia,70,mortes por coronavirus no reino unido por regiao,70


In [8]:
with open('./avaliador_backup.json') as infile:
    lista = json.load(infile)

In [11]:
df_2 = pd.DataFrame(lista,columns=['query','per_um','ava_um','per_dois','ava_dois','per_tres','ava_tres'])
df_2

,query,per_um,ava_um,per_dois,ava_dois,per_tres,ava_tres
0,o que e periodo de incubacao cobicado,o que e o periodo de incubacao de covid-19s?,50,o que e o periodo de incubacao do coronavirus,70,"o que e o ""periodo de incubacao""?",80
1,covid-19 realmente se originou da provincia da...,onde covid-19 se originou?,80,a taxa de recuperacao muito alta da china de c...,0,a administracao trump realmente herdou um test...,0
2,como o coronavirus se compara a praga,como o coronavirus se compara,50,como o coronavirus se compara a gripe suina,0,como o covid-19 se compara a outros virus?,40
3,o que torna meu sistema imunologico mais forte,o que torna seu sistema imunologico mais forte,100,que vitamina torna o seu sistema imunologico m...,40,beijar torna seu sistema imunologico mais forte,30
4,ouvi falar que a cloroquina e hidroxicloroquin...,o que devo fazer se ficar doente durante uma v...,30,quando eu devo usar luvas?,0,o que devo fazer se alguem da minha familia fi...,30
...,...,...,...,...,...,...,...
139,pode baixo sistema imunologico causar infeccao...,pode baixo sistema imunologico causar urticaria,0,pode baixo sistema imunologico causar infertil...,0,pode baixo sistema imunologico causar aborto,0
140,o desinfetante para as maos e util,pode o desinfetante caseiro substituir desinfe...,80,posso matar covid-19 sem desinfetante para as ...,80,posso usar vodka como desinfetante para as maos?,0
141,como voce estimula um sistema imunologico fraco,como voce estimula o sistema imunologico de um...,0,como voce fortalece um sistema imunologico fraco,100,como voce tem um sistema imunologico fraco,0
142,o que posso fazer para impulsionar meu sistema...,o que posso fazer para impulsionar meu sistema...,65,o que posso fazer para impulsionar o sistema i...,70,o que posso fazer para manter meu sistema imun...,70


In [12]:
result = pd.concat([df,df_2])
result.reset_index(drop=True,inplace=True)
result

,query,per_um,ava_um,per_dois,ava_dois,per_tres,ava_tres
0,o covid-19 acabara com o mundo,"assumindo que covid-19 nao acabou, isso acabar...",100,o covid-19 algum dia acabara?,0,que licoes o mundo aprendera com covid-19?,0
1,o que e covid 19 malaysia,covid 19 malaysia puchong,100,covid 19 malaysia berita,100,covid 19 malaysia moh,100
2,onde esta o coronavirus reino unido,onde esta o coronavirus agora no reino unido,100,"coronavirus no reino unido, onde",100,casos de coronavirus no reino unido onde,100
3,como as familias podem se preparar para o bloq...,como se preparar para o coronavirus nos eua,0,o que os idosos podem fazer para se preparar p...,0,como a oms esta ajudando os paises a se prepar...,0
4,quantos casos de coronavirus na fl,quantos casos de coronavirus em orlando fl,100,quantos casos de coronavirus em jacksonville fl,100,quantos casos de coronavirus em napoles fl,100
...,...,...,...,...,...,...,...
495,pode baixo sistema imunologico causar infeccao...,pode baixo sistema imunologico causar urticaria,0,pode baixo sistema imunologico causar infertil...,0,pode baixo sistema imunologico causar aborto,0
496,o desinfetante para as maos e util,pode o desinfetante caseiro substituir desinfe...,80,posso matar covid-19 sem desinfetante para as ...,80,posso usar vodka como desinfetante para as maos?,0
497,como voce estimula um sistema imunologico fraco,como voce estimula o sistema imunologico de um...,0,como voce fortalece um sistema imunologico fraco,100,como voce tem um sistema imunologico fraco,0
498,o que posso fazer para impulsionar meu sistema...,o que posso fazer para impulsionar meu sistema...,65,o que posso fazer para impulsionar o sistema i...,70,o que posso fazer para manter meu sistema imun...,70


In [20]:
result.to_csv('./gabarito_parcial.csv',index=False)

# Merge resultados avaliados etapa 3

In [21]:
df = pd.read_csv('./gabarito_parcial.csv')
df

,query,per_um,ava_um,per_dois,ava_dois,per_tres,ava_tres
0,o covid-19 acabara com o mundo,"assumindo que covid-19 nao acabou, isso acabar...",100,o covid-19 algum dia acabara?,0,que licoes o mundo aprendera com covid-19?,0
1,o que e covid 19 malaysia,covid 19 malaysia puchong,100,covid 19 malaysia berita,100,covid 19 malaysia moh,100
2,onde esta o coronavirus reino unido,onde esta o coronavirus agora no reino unido,100,"coronavirus no reino unido, onde",100,casos de coronavirus no reino unido onde,100
3,como as familias podem se preparar para o bloq...,como se preparar para o coronavirus nos eua,0,o que os idosos podem fazer para se preparar p...,0,como a oms esta ajudando os paises a se prepar...,0
4,quantos casos de coronavirus na fl,quantos casos de coronavirus em orlando fl,100,quantos casos de coronavirus em jacksonville fl,100,quantos casos de coronavirus em napoles fl,100
...,...,...,...,...,...,...,...
495,pode baixo sistema imunologico causar infeccao...,pode baixo sistema imunologico causar urticaria,0,pode baixo sistema imunologico causar infertil...,0,pode baixo sistema imunologico causar aborto,0
496,o desinfetante para as maos e util,pode o desinfetante caseiro substituir desinfe...,80,posso matar covid-19 sem desinfetante para as ...,80,posso usar vodka como desinfetante para as maos?,0
497,como voce estimula um sistema imunologico fraco,como voce estimula o sistema imunologico de um...,0,como voce fortalece um sistema imunologico fraco,100,como voce tem um sistema imunologico fraco,0
498,o que posso fazer para impulsionar meu sistema...,o que posso fazer para impulsionar meu sistema...,65,o que posso fazer para impulsionar o sistema i...,70,o que posso fazer para manter meu sistema imun...,70


In [22]:
df_copy = df[df['ava_um'] == 0]
df_copy = df_copy[df_copy['ava_dois'] == 0]
df_copy = df_copy[df_copy['ava_tres'] == 0]

In [24]:
df_copy.index

Int64Index([  3,   7,  32,  33,  34,  49,  54,  72,  73,  76,  78,  81,  85,
             86,  89,  92, 101, 117, 118, 129, 131, 134, 139, 141, 146, 159,
            166, 167, 173, 176, 177, 191, 194, 217, 222, 228, 241, 245, 249,
            277, 285, 290, 296, 301, 309, 311, 312, 314, 322, 326, 328, 338,
            347, 349, 379, 386, 387, 390, 394, 402, 403, 426, 430, 437, 442,
            445, 452, 462, 464, 465, 467, 469, 471, 489, 492, 495],
           dtype='int64')

In [56]:
# 'como as familias podem se preparar para o bloqueio'
for each_list in document:
    if(('africa') in each_list):
        if('sul' in each_list):
            print(' '.join(each_list))

sobre o virus corona na africa do sul
casos de virus corona na africa do sul
casos de coronavirus na africa do sul
virus corona e africa do sul
virus corona na africa do sul
virus corona na africa do sul mais recente
virus corona nas noticias da africa do sul
virus corona da africa do sul mais recente
noticias da africa do sul virus corona
mortes por coronavirus na africa do sul
coronavirus em casos da africa do sul
coronavirus na africa do sul casos confirmados
coronavirus nas noticias da africa do sul
coronavirus nas noticias da africa do sul 24
coronavirus nos numeros da africa do sul
coronavirus nas estatisticas da africa do sul
coronavirus nos sintomas da africa do sul
coronavirus e africa do sul
ultimas noticias sobre coronavirus africa do sul
atualizacao de coronavirus africa do sul
atualizacao de coronavirus estado livre da africa do sul
quantos casos de coronavirus na africa do sul
e o virus corona na africa do sul
o virus corona esta agora na africa do sul
e coronavirus na af

In [33]:
document.size

12239